In [1]:
import os
import json
from pyspark.sql import SparkSession

os.environ["HADOOP_HOME"] = r"C:\hadoop"
os.environ["PATH"] += os.pathsep + r"C:\hadoop\bin"
os.environ['JAVA_HOME'] = r"D:\Applns\Amazon Corretto\jdk17.0.15_6"
os.environ['PATH'] = r"D:\Applns\Amazon Corretto\jdk17.0.15_6\bin" + os.pathsep + os.environ['PATH']


In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("RumbleVideoETL_silverlayer") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()


In [3]:
bronze_data = spark.read.parquet("../data/bronze/rumble_videos_bronze.parquet")
selected_data = bronze_data.select("video_id", "video_url", "video_title", "video_host", "video_duration_s",
    "video_date", "video_views", "upvotes_count", "downvotes_count",
    "video_description", "video_tags", "comments")

silver_layer_data = selected_data.repartition(5).write.mode("overwrite").parquet("../data/silver")


In [4]:
read_silver_layer_data  = spark.read.parquet("../data/silver")
read_silver_layer_data.show()


+--------+--------------------+--------------------+-----------------+----------------+--------------------+-----------+-------------+---------------+--------------------+--------------------+--------------------+
|video_id|           video_url|         video_title|       video_host|video_duration_s|          video_date|video_views|upvotes_count|downvotes_count|   video_description|          video_tags|            comments|
+--------+--------------------+--------------------+-----------------+----------------+--------------------+-----------+-------------+---------------+--------------------+--------------------+--------------------+
| v5c48et|https://rumble.co...|Is Trump Winning ...|         Tim Pool|            7823| 2024-08-23 00:00:00|       3970|           61|              4|     Host:\nTim Pool|[Entertainment, t...|[{0, You're not g...|
| v5d06le|https://rumble.co...|Creationism vs Si...|         Tim Pool|            7581|2024-08-30T21:00:...|      48900|          126|          

In [6]:
spark.stop()